#### 几个参考链接

* https://velocity.apache.org/engine/1.7/user-guide.html#what-is-velocity
* https://blogs.cisco.com/developer/using-velocity-to-create-plug-and-play-templates-in-dna-center-part-4
* https://github.com/purcell/airspeed/
* https://github.com/cisco-en-programmability/dnacentersdk
* https://thisbridgeistheroot.com/blog/dna-center-advanced-scripting

* https://guides.github.com/features/mastering-markdown/

#### 如何使用airspeed，也就是如何更好地使用DNA center template

In [2]:
import airspeed

In [24]:
# 使用set在模板中直接赋值
# 描述行使用##
t = airspeed.Template("""
## one line comments only 提示行，程序不运行

#*
  多行描述方法，本段也不运行
*#

#set ($hostname = "switch1")
hostname $hostname
""")

print(t.merge({}))
# {} 空表示不需要数据传递


hostname switch1



In [10]:
# 在渲染时，赋值传递至模板，python常用这个方法
t = airspeed.Template("""

##set ($hostname = "switch1")
hostname $hostname
""")

print(t.merge({"hostname": "switch2"}))


hostname switch2



In [13]:
# 变量范围需要标记 Formal Notation
t = airspeed.Template("""

##set ($hostname = "switch1")
hostname $hostnameshanghai
hostname ${hostname}shanghai
""")

print(t.merge({"hostname": "switch2"}))


hostname $hostnameshanghai
hostname switch2shanghai



In [18]:
# array， python中就是list
t = airspeed.Template("""
#set ($DHCPAddresses = ["10.1.1.100", "10.1.1.101", "10.1.2.100", "10.1.2.101"])
#foreach ($address in $DHCPAddresses)
 ip helper-address $address
#end
""")

print(t.merge({}))



 ip helper-address 10.1.1.100

 ip helper-address 10.1.1.101

 ip helper-address 10.1.2.100

 ip helper-address 10.1.2.101



In [23]:
# array， python中就是list，变量传递给模板
t1 = airspeed.Template("""
#foreach ($address in $DHCPAddresses)
 ip helper-address $address
#end
""")

dhcp_list = ["10.1.1.100", "10.1.1.101", "10.1.2.100", "10.1.2.101"]
print(t1.merge({"DHCPAddresses": dhcp_list}))



 ip helper-address 10.1.1.100

 ip helper-address 10.1.1.101

 ip helper-address 10.1.2.100

 ip helper-address 10.1.2.101



In [27]:
# foreach 端口循环，数字表达式
t = airspeed.Template("""
#set ($stack = 3)
#foreach ($switch in [1..$stack])
interface range GigabitEthernet$switch/0/1-48
 description User Port
 switchport mode access
 switchport access vlan 10
!
#end
""")

print(t.merge({}))



interface range GigabitEthernet1/0/1-48
 description User Port
 switchport mode access
 switchport access vlan 10
!

interface range GigabitEthernet2/0/1-48
 description User Port
 switchport mode access
 switchport access vlan 10
!

interface range GigabitEthernet3/0/1-48
 description User Port
 switchport mode access
 switchport access vlan 10
!



In [29]:
# foreach list中序号作为变量赋值
t = airspeed.Template("""
#set ($subnets = ["10.1.1.1", "10.1.2.1", "10.1.3.1", "10.1.4.1"])
#foreach ($address in $subnets)
vlan 10$foreach.index
 name Users_$address
interface vlan 10$foreach.index
 ip address $address 255.255.255.0
#end
""")

print(t.merge({}))



vlan 100
 name Users_10.1.1.1
interface vlan 100
 ip address 10.1.1.1 255.255.255.0

vlan 101
 name Users_10.1.2.1
interface vlan 101
 ip address 10.1.2.1 255.255.255.0

vlan 102
 name Users_10.1.3.1
interface vlan 102
 ip address 10.1.3.1 255.255.255.0

vlan 103
 name Users_10.1.4.1
interface vlan 103
 ip address 10.1.4.1 255.255.255.0



In [39]:
# macro 重复块使用
t = airspeed.Template("""
#macro(distribution_interface)
 switchport
 switchport mode trunk
 switchport trunk allowed vlan 100-399
 switchport trunk native vlan 100
 storm-control broadcast level .4
 channel-group 1 mode active 
#end

interface TenGigabitEthernet1/0/8
#distribution_interface()

interface TenGigabitEthernet1/0/9
#distribution_interface()
""")

print(t.merge({}))



interface TenGigabitEthernet1/0/8
 switchport
 switchport mode trunk
 switchport trunk allowed vlan 100-399
 switchport trunk native vlan 100
 storm-control broadcast level .4
 channel-group 1 mode active 


interface TenGigabitEthernet1/0/9
 switchport
 switchport mode trunk
 switchport trunk allowed vlan 100-399
 switchport trunk native vlan 100
 storm-control broadcast level .4
 channel-group 1 mode active 




In [40]:
# macro 重复块使用中，使用变量赋值
t = airspeed.Template("""
#macro(access_interface $vlan)
 switchport mode access
 switchport access vlan $vlan
 spanning-tree portfast
 spanning-tree bpduguard enable
#end

interface gigabitethernet1/0/1
#access_interface(10)


interface gigabitethernet1/0/2
#access_interface(10)

""")

print(t.merge({}))



interface gigabitethernet1/0/1
 switchport mode access
 switchport access vlan 10
 spanning-tree portfast
 spanning-tree bpduguard enable



interface gigabitethernet1/0/2
 switchport mode access
 switchport access vlan 10
 spanning-tree portfast
 spanning-tree bpduguard enable





In [43]:
# If else end
t = airspeed.Template("""
#set ($stack = 4)
#if( $stack > 1 )
interface range TenGigabitEthernet1/1/8,TenGigabitEthernet2/1/8
#else
interface range TenGigabitEthernet1/1/7-8
#end 
 Description Uplink to Distribution Switch 
 switchport mode trunk 
 switchport trunk allowed vlan 100-200
 channel-group 1 mode active
""")

print(t.merge({}))


interface range TenGigabitEthernet1/1/8,TenGigabitEthernet2/1/8
 Description Uplink to Distribution Switch 
 switchport mode trunk 
 switchport trunk allowed vlan 100-200
 channel-group 1 mode active



In [50]:
# FUNction
t = airspeed.Template("""
#set ($usernamepws = ["admin/cisco123", "superuser/password123"])
#foreach ($user in $usernamepws)
#set ($split = $user.split("/"))
username $split[0] privilege 15 algorithm-type scrypt secret $split[1]
#end
""")

print(t.merge({}))



username admin privilege 15 algorithm-type scrypt secret cisco123

username superuser privilege 15 algorithm-type scrypt secret password123



In [49]:
# 计算方法
t = airspeed.Template("""
#set( $APstart = 49 - $APinterfaces )
interface range GigabitEthernet1/0/$APstart-48
 description Access Point
 switchport mode access
 switchport access vlan 20
""")

print(t.merge({"APinterfaces": 4}))


interface range GigabitEthernet1/0/45-48
 description Access Point
 switchport mode access
 switchport access vlan 20

